In [16]:
from sumpy.recurrence import _make_sympy_vec

from sumpy.expansion.diff_op import (
    laplacian,
    make_identity_diff_op,
)

from sumpy.recurrence import _generate_nd_derivative_relations, pde_to_ode_in_r, ode_in_r_to_x, recurrence_from_pde,ode_in_x_to_coeff_array, recurrence_from_coeff_array

import sympy as sp
from sympy import hankel1

import numpy as np

import math

import matplotlib.pyplot as plt
from matplotlib import cm, ticker

In [17]:
from sumpy.expansion.diff_op import DerivativeIdentifier, LinearPDESystemOperator
from immutabledict import immutabledict

In [18]:
w = make_identity_diff_op(2)
laplace2d = laplacian(w)
g = sp.Function('g')

In [19]:
var = _make_sympy_vec("x", 3)
rels = _generate_nd_derivative_relations(var, 4)

In [20]:
from collections import namedtuple
DerivativeIdentifier = namedtuple("DerivativeIdentifier", ["mi", "vec_idx"])
partial_1 = DerivativeIdentifier((1,0,3), 0)
list_pde = {partial_1: 1}
list_pde = immutabledict(list_pde)
pde_op = LinearPDESystemOperator(3,(list_pde,))
pde_op

LinearPDESystemOperator(3, (immutabledict({DerivativeIdentifier(mi=(1, 0, 3), vec_idx=0): 1}),))

In [21]:
ode_in_r, var, ode_order = pde_to_ode_in_r(pde_op)
ode_in_r

x0*x2*(-3*f_r1*(5*x2**2/(x0**2 + x1**2 + x2**2) - 3)/(x0**2 + x1**2 + x2**2)**(5/2) + 3*f_r2*(3*x2**2/(x0**2 + x1**2 + x2**2) - 1)/(x0**2 + x1**2 + x2**2)**2 + (3*f_r2*x2**2/(x0**2 + x1**2 + x2**2)**(5/2) - 3*f_r2/(x0**2 + x1**2 + x2**2)**(3/2) - 3*f_r3*x2**2/(x0**2 + x1**2 + x2**2)**2 + 3*f_r3/(x0**2 + x1**2 + x2**2) + f_r4*x2**2/(x0**2 + x1**2 + x2**2)**(3/2))/sqrt(x0**2 + x1**2 + x2**2) - 3*(-f_r2*x2**2/(x0**2 + x1**2 + x2**2)**(3/2) + f_r2/sqrt(x0**2 + x1**2 + x2**2) + f_r3*x2**2/(x0**2 + x1**2 + x2**2))/(x0**2 + x1**2 + x2**2)**(3/2))

In [22]:
f_x_derivs = _make_sympy_vec("f_x", ode_order+1)
ode_in_x = ode_in_r.subs(rels)

In [10]:
ode_in_x.simplify()

x2*(9*f_x1*x0**2 - 15*f_x1*x2**2 - 9*f_x2*x0**3 + 15*f_x2*x0*x2**2 + 3*f_x3*x0**4 - 6*f_x3*x0**2*x2**2 + f_x4*x0**3*x2**2)/x0**6

In [11]:
sp.poly(ode_in_x, list(f_x_derivs)).coeffs()[0].simplify()

(9*x0**2*x2 - 15*x2**3)/x0**6

In [33]:
def generate_partitions(n):
    """
    Generates all integer partitions of n.

    Args:
        n: The integer to partition.

    Returns:
        A list of lists, where each inner list is a partition of n.
    """
    if n == 0:
        return [[]]
    
    partitions = []
    for i in range(1, n + 1):
        for p in generate_partitions(n - i):
            if not p or i <= p[0]:
                temp = [i] + p
                if np.all(np.array(temp) <= 2):
                    partitions.append([i] + p)
    return partitions

In [74]:
nx = 3
ny = 3
all_nx = generate_partitions(nx)
all_ny = generate_partitions(ny)

In [75]:
r = var[0]**2 + var[1]**2

In [76]:
sum = np.zeros(nx+ny+1) * var[0]

In [77]:
for i_x in all_nx:
    for i_y in all_ny:
        k = len(i_x) + len(i_y)
        prod = 1
        for i in i_x:
            prod *= sp.diff(r, var[0], i)
        for j in i_y:
            prod *= sp.diff(r, var[1], j)
        sum[k] += prod

In [78]:
sum

array([0, 0, 0, 0, 16*x0*x1, 32*x0**3*x1 + 32*x0*x1**3, 64*x0**3*x1**3],
      dtype=object)

In [82]:
factors = sum/np.array([r**(2*i) for i in range(len(sum))])
factors

array([0, 0, 0, 0, 16*x0*x1/(x0**2 + x1**2)**8,
       (32*x0**3*x1 + 32*x0*x1**3)/(x0**2 + x1**2)**10,
       64*x0**3*x1**3/(x0**2 + x1**2)**12], dtype=object)